### This program will generate the Y category to feed into the algorithm
Buy
Record buy price bp
Record buy -5% as bl
	if price < bl
	 sell
	 record date, buy and sell
	if price > bl
 	 bl=price -5%

sell
sell price as sp
sl = sp + 5%
	if price > sl
	 buy
	 record buy
	if price < sl


It is implementing what is called a trailing stop loss	 sl = price +5%

In [2]:
import pandas as pd


In [3]:
df = pd.read_csv('SPY_Daily.csv')
df.head()

,Date Time,Open,High,Low,Close,Change,200 Moving Average,100 Moving Average,50 Moving Average,Bollinger-Upper,Bollinger-Mid,Bollinger-Lower,RSI,RSI-Average,MACD,MACD-Signal,MACD-Histogram,Volume
0,02/05/2022,412.07,415.92,405.0200,414.48,2.48,448.06405,447.4625,436.7374,461.656903,435.8995,410.142097,37.173459,36.455153,-11.623462,-7.360043,-4.263419,158312500
1,03/05/2022,415.01,418.93,413.3600,416.38,1.90,448.02110,446.9311,436.3804,459.096391,433.8785,408.660609,38.708659,37.642485,-12.011090,-8.516318,-3.494772,100028100
2,04/05/2022,417.08,429.66,413.7099,429.06,12.68,448.01110,446.5582,436.3702,456.799243,432.7800,408.760757,47.864633,40.433830,-11.720897,-9.416019,-2.304879,144247800
3,05/05/2022,424.55,425.00,409.4400,413.81,-15.25,447.90740,445.9889,436.2074,455.648362,431.1445,406.640638,40.105029,39.811642,-12.487244,-10.210420,-2.276823,172929100
4,06/05/2022,411.10,414.80,405.7300,411.34,-2.47,447.78680,445.4366,435.8682,453.823788,429.2730,404.722212,39.002157,40.570787,-13.446538,-10.998326,-2.448212,151770800


In [4]:
#Setting up the action column
df['Act']='SELL'
df.head()

,Date Time,Open,High,Low,Close,Change,200 Moving Average,100 Moving Average,50 Moving Average,Bollinger-Upper,Bollinger-Mid,Bollinger-Lower,RSI,RSI-Average,MACD,MACD-Signal,MACD-Histogram,Volume,Act
0,02/05/2022,412.07,415.92,405.0200,414.48,2.48,448.06405,447.4625,436.7374,461.656903,435.8995,410.142097,37.173459,36.455153,-11.623462,-7.360043,-4.263419,158312500,SELL
1,03/05/2022,415.01,418.93,413.3600,416.38,1.90,448.02110,446.9311,436.3804,459.096391,433.8785,408.660609,38.708659,37.642485,-12.011090,-8.516318,-3.494772,100028100,SELL
2,04/05/2022,417.08,429.66,413.7099,429.06,12.68,448.01110,446.5582,436.3702,456.799243,432.7800,408.760757,47.864633,40.433830,-11.720897,-9.416019,-2.304879,144247800,SELL
3,05/05/2022,424.55,425.00,409.4400,413.81,-15.25,447.90740,445.9889,436.2074,455.648362,431.1445,406.640638,40.105029,39.811642,-12.487244,-10.210420,-2.276823,172929100,SELL
4,06/05/2022,411.10,414.80,405.7300,411.34,-2.47,447.78680,445.4366,435.8682,453.823788,429.2730,404.722212,39.002157,40.570787,-13.446538,-10.998326,-2.448212,151770800,SELL


In [5]:

stopLoss=0
actPrice=0
lastAct='SELL'
margin =0.02

def slact(dfRow):
    global lastAct, stopLoss, actPrice, margin
    price=dfRow.Close
    if lastAct == 'BUY':        
        if price < stopLoss:
            lastAct='SELL'
            stopLoss = price + (margin * price)
            actPrice=price
            
        elif price > actPrice and price - (margin * price) > stopLoss:
            stopLoss = price - (margin * price)
            
    elif lastAct == 'SELL':
        if price > stopLoss:
            lastAct='BUY'
            
            stopLoss = price - (margin * price)
            actPrice=price
            
        elif price < actPrice and price + (margin * price) < stopLoss:
            stopLoss = price + (margin * price)
            
    return lastAct

In [6]:
"""
for each row of df
    out=act(Close)
    append out to a 1d array

"""
df['Act']=df.apply(slact,axis=1)
out=df['Act']
out.to_csv('Actions.csv', index=False)

In [7]:
df.head()

,Date Time,Open,High,Low,Close,Change,200 Moving Average,100 Moving Average,50 Moving Average,Bollinger-Upper,Bollinger-Mid,Bollinger-Lower,RSI,RSI-Average,MACD,MACD-Signal,MACD-Histogram,Volume,Act
0,02/05/2022,412.07,415.92,405.0200,414.48,2.48,448.06405,447.4625,436.7374,461.656903,435.8995,410.142097,37.173459,36.455153,-11.623462,-7.360043,-4.263419,158312500,BUY
1,03/05/2022,415.01,418.93,413.3600,416.38,1.90,448.02110,446.9311,436.3804,459.096391,433.8785,408.660609,38.708659,37.642485,-12.011090,-8.516318,-3.494772,100028100,BUY
2,04/05/2022,417.08,429.66,413.7099,429.06,12.68,448.01110,446.5582,436.3702,456.799243,432.7800,408.760757,47.864633,40.433830,-11.720897,-9.416019,-2.304879,144247800,BUY
3,05/05/2022,424.55,425.00,409.4400,413.81,-15.25,447.90740,445.9889,436.2074,455.648362,431.1445,406.640638,40.105029,39.811642,-12.487244,-10.210420,-2.276823,172929100,SELL
4,06/05/2022,411.10,414.80,405.7300,411.34,-2.47,447.78680,445.4366,435.8682,453.823788,429.2730,404.722212,39.002157,40.570787,-13.446538,-10.998326,-2.448212,151770800,SELL
